In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import geopy.distance
from shapely.geometry import Point, Polygon, LineString
import pyproj
import shapely.ops

In [ ]:
map = gpd.read_file("geo_data/melbourne.shp")
polygon = map.iloc[0][1]

In [ ]:
gda94 = pyproj.CRS('EPSG:4283')
proj = pyproj.CRS('EPSG:28355')
project = pyproj.Transformer.from_crs(gda94, proj, always_xy=True).transform

In [ ]:
def waterfront_filter(coord):
    return coord[0] < 145.15 and coord[0] > 144.6 and coord[1] < -37.8 and coord[1] > -38.3

seq = list(filter(waterfront_filter,polygon.exterior.coords))
ls = LineString(seq)
lsp = shapely.ops.transform(project,ls)

In [ ]:
def compute_waterfront_dist(df):
    df["location"] = df.apply(lambda h : Point((h.long,h.Lat)),axis=1)
    df["locationp"] = df.location.transform(lambda point : shapely.ops.transform(project,point))
    wfdist = df.locationp.transform(lambda pp : lsp.distance(pp))
    return wfdist

In [ ]:
X_train = pd.read_csv("data/X_train.csv", index_col=0)
X_test = pd.read_csv("data/X_test.csv", index_col=0)
compute_waterfront_dist(X_train).to_csv("features/waterfront_dist_train.csv")
compute_waterfront_dist(X_test).to_csv("features/waterfront_dist_test.csv")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e6ac642c-b46b-41ed-b99b-6fa6e21df87a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>